In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from evaluation.utils import entropy_batch_mixing, knn_purity
import scanpy as sc
import seaborn as sns
import scIB as scib
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_datasets/pancreas_normalized.h5ad'))
batch_key = 'study'
label_key = 'cell_type'
ratios = [1, 2]

In [5]:
def compute_metrics(latent_adata, adata, rqr=None, batch_key='study', label_key='cell_type'):
    latent_adata.obsm['X_pca'] = latent_adata.X
    print(adata.shape, latent_adata.shape)
    n_batches = len(adata.obs[batch_key].unique().tolist())
    
    scores = scib.metrics.metrics(adata, latent_adata, batch_key, label_key, 
                                  nmi_=True, ari_=True, silhouette_=True, pcr_=True, graph_conn_=True, 
                                  isolated_labels_=True, hvg_score_=False)
    scores = scores.T
    scores = scores[['NMI_cluster/label', 'ARI_cluster/label', 'ASW_label', 'ASW_label/batch', 
                     'PCR_batch', 'isolated_label_F1', 'isolated_label_silhouette', 'graph_conn']]
    
    ebm = entropy_batch_mixing(latent_adata, batch_key, n_neighbors=15)
    knn = knn_purity(latent_adata, label_key, n_neighbors=15)
    
    scores['EBM'] = ebm
    scores['KNN'] = knn
    scores['method'] = 'scvi'
    scores['data'] = 'pancreas'
    scores['rqr'] = rqr / n_batches if rqr is not None else None
    
    return scores

## Calculating metrics for all ratios of pancreas scvi:

In [6]:
scores = None
for ratio in ratios:
    test_num = ratio
    latent_adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_adata/figure_3/scvi/pancreas/test_{test_num}_first_cond/full_data.h5ad'))
    latent_adata.obs[batch_key] = latent_adata.obs['batch'].values
    latent_adata.obs[label_key] = latent_adata.obs['celltype'].values  
    df = compute_metrics(latent_adata, adata, ratio, batch_key, label_key)
    scores = pd.concat([scores, df], axis=0) if scores is not None else df                

AnnData object with n_obs × n_vars = 15681 × 10
    obs: 'celltype', 'batch'
(15681, 1000) (15681, 10)
clustering...
NMI...
ARI...
silhouette score...
PC regression...
isolated labels...
Graph connectivity...
Calculating EBM with n_cat = 5


/home/marco/.pyenv/versions/3.7.7/envs/sca_testing/lib/python3.7/site-packages/scIB/metrics.py:1746: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  _,labs = connected_components(adata_post_sub.uns['neighbors']['connectivities'], connection='strong')


EBM: 0.2651068478092024
KNN-P: 0.8534394567493742
AnnData object with n_obs × n_vars = 15681 × 10
    obs: 'celltype', 'batch'
(15681, 1000) (15681, 10)
clustering...
NMI...
ARI...
silhouette score...
PC regression...
isolated labels...
Graph connectivity...
Calculating EBM with n_cat = 5


/home/marco/.pyenv/versions/3.7.7/envs/sca_testing/lib/python3.7/site-packages/scIB/metrics.py:1746: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  _,labs = connected_components(adata_post_sub.uns['neighbors']['connectivities'], connection='strong')


EBM: 0.2588008993179193
KNN-P: 0.8562700007810556


In [7]:
scores

,NMI_cluster/label,ARI_cluster/label,ASW_label,ASW_label/batch,PCR_batch,isolated_label_F1,isolated_label_silhouette,graph_conn,EBM,KNN,method,data,rqr
0,0.703527,0.625275,0.578273,0.867146,0.303744,0.837794,0.548268,0.987036,0.265107,0.853439,scvi,pancreas,0.2
0,0.734435,0.731526,0.575940,0.875444,0.606740,0.844173,0.547147,0.990914,0.258801,0.856270,scvi,pancreas,0.4


In [8]:
scores.to_csv(os.path.expanduser(f'~/Documents/rqr_pancreas_scvi_1_2.csv'), index=False)

In [9]:
df = pd.read_csv(os.path.expanduser(f'~/Documents/rqr_pancreas_scvi_1_2.csv'))
df['reference_time'] = 0.0
df['query_time'] = 0.0
df.rqr = df.rqr.round(2)
df.to_csv(os.path.expanduser(f'~/Documents/rqr_pancreas_scvi_1_2_final.csv'), index=False)